# Data 605 Final Exam: Problem 2
### David Blumenstiel

In [ ]:
#lets get this one out of the way first
library(tidyverse)
library(psych)
library(MASS)

**5 points.  Descriptive and Inferential Statistics. Provide univariate descriptive statistics and appropriate plots for the training data set.  Provide a scatterplot matrix for at least two of the independent variables and the dependent variable. Derive a correlation matrix for any three quantitative variables in the dataset.  Test the hypotheses that the correlations between each pairwise set of variables is 0 and provide an 80% confidence interval.  Discuss the meaning of your analysis.  Would you be worried about familywise error? Why or why not?**

First thing we need to do is load in the data.  We'll reserve the testing set for later, and just load in the training set as 'df' for now.  Let's also take a look at the shape of the data along with univariate descriptive statistics for each variable.


In [ ]:
df <- read.csv('../input/house-prices-advanced-regression-techniques/train.csv')

dim(df)
summary(df)

As you can see, there are quite a few variables; 81 of them to be exact.  There are 1460 observations in this training set, which should be ample for our purposes.

I'm not going to go into the details of every variable above, but you can see there are a few different types of data.  We have categorical data, continuous numeric data, and discrete numerical data (some of which would be better off as categorical data).  There's some stuff like 'ID' that has little bearing, and missing data which could be coerced to more appropriate values.  

Let's take a more in depth look at the response variable, 'SalePrice'.


In [ ]:
summary(df$SalePrice)
hist(df$SalePrice)

As one might expect, prices are somewhat right skewed, with most houses being worth around $130,000 to 214,000 with some houses worth significantly more.

Now we'll do a scatterplot/correlation matrix for a few of the independent variables and the dependent variable.  We'll choose "SalePrice","LotArea","YearBuilt","FullBath","OpenPorchSF","GrLivArea","TotalBsmtSF".  These ones have some reasonable correlation to the sale price.

In [ ]:
colnames(df)

In [ ]:
pairs.panels(df[,c("SalePrice","LotArea","YearBuilt","FullBath","OpenPorchSF","GrLivArea","TotalBsmtSF")])

Above we have the pairs plots containing correlations and scatters.  Some things that stand out are that the above ground living room square area, 'GrLivArea', is the most highly correlated dependent variable of the set to 'SalePrice' with a correlation coefficient of 0.71, and lot area is the least correlated with a coefficient of 0.26.  This makes some sense, as greater living area makes for bigger houses, and smaller lots tend to include things like city homes/apartments, which may be expensive despite having small lots.

Some other interesting facets: the newer the house, the more full-bathrooms and basement space; lot area has little effect on how big the porch gets; the number of full bathrooms are highly correlated to the amount of living space.

For now, we'll cut this down to three variables as requested: "SalePrice","FullBath" and "GrLivArea".





In [ ]:
pairs.panels(df[,c("SalePrice","FullBath","GrLivArea")])

Now we'll test the hypothesis that the correlations between each pairwise set is 0, and provide an 80% confidence interval.

In [ ]:
cor.test(df$SalePrice, df$FullBath, conf.level = 0.8)
cor.test(df$SalePrice, df$GrLivArea, conf.level = 0.8)
cor.test(df$FullBath, df$GrLivArea, conf.level = 0.8)

Above are the three correlation tests, one for each pair.  The hypothesis for all three goes as: if the p-value is less than the significance level, then the correlation is not 0 (there is a correlation).  The p-value for all three pairs is pretty close to 0, so at the standard significance level of 0.05, we can conclude that there is indeed a correlation between each of the variables and one-another individually.  

When it comes to the correlation coefficients themselves, all are estimated to be positive, indicating potential positive correlations (if x increases, y increases ) among all pairs.  The 80% confidence intervals can be interpreted as: "We are 80% confident that the true correlation coefficient falls within this range."  If that range excludes 0, we can further be 80% sure that there are indeed correlations here,

Familywise error here is the likelihood that at least one of correlation findings were false; i.e., there isn't a correlation between one of the pairs.  Given how low the p-values are, and that there are only three correlations, I'm not worried about familywise error.  In fact, we can quantify the familywise error rate as: $FWE \leq 1 - (1-\alpha)^c$ where $\alpha$ is the significance level, and c is the number of tests performed.  Because the p-values are near 0, we could have set the significance level to something stupid like 0.00001, and had a very small familywise error rate.

On an interesting side note, if you think about how scientific literature tends to go by a 0.05 significance level, you kind of start to get an idea of just how much stuff out there probably isn't right.


**5 points. Linear Algebra and Correlation.  Invert your correlation matrix from above. (This is known as the precision matrix and contains variance inflation factors on the diagonal.) Multiply the correlation matrix by the precision matrix, and then multiply the precision matrix by the correlation matrix. Conduct LU decomposition on the matrix. **

Below, we'll give the correlation matrix from before (in actual matrix form this time) along with the precision matrix.  We'll also multiply the two together (both ways)

In [ ]:
cormat <- cor(x=df[c("SalePrice","FullBath","GrLivArea")]) #Gotta actually make the correlation matrix as a matrix first

cormat

permat <- solve(cormat) #The percision matrix

permat

cormat_by_permat <- cormat %*% permat #correlation matrix * percision matrix

cormat_by_permat

permat_by_cormat <- permat %*% cormat #percision matrix * correlation matrix

permat_by_cormat

Now we'll perform LU decomposition on the correlation matrix.  The 'pracma' package has a nice function for this.

In [ ]:
#LU Decomposition of the correlation matrix
library(pracma)
LUDecomp_cormat <- lu(cormat)
LUDecomp_cormat

**5 points.  Calculus-Based Probability & Statistics.  Many times, it makes sense to fit a closed form distribution to data.  Select a variable in the Kaggle.com training dataset that is skewed to the right, shift it so that the minimum value is absolutely above zero if necessary.  Then load the MASS package and run fitdistr to fit an exponential probability density function.  (See  https://stat.ethz.ch/R-manual/R-devel/library/MASS/html/fitdistr.html ).  Find the optimal value of $\lambda$ for this distribution, and then take 1000 samples from this exponential distribution using this value (e.g., rexp(1000,$\lambda$ )).  Plot a histogram and compare it with a histogram of your original variable.   Using the exponential pdf, find the 5th and 95th percentiles using the cumulative distribution function (CDF).   Also generate a 95% confidence interval from the empirical data, assuming normality.  Finally, provide the empirical 5th percentile and 95th percentile of the data.  Discuss.**

If you remember, our response variable, 'SalePrice', is right skewed; we'll use that.  

In [ ]:
saleprice <- df$SalePrice #Makes it easier to call

if (min(saleprice) <= 0) { #Checks if the min is 0; it isn't
    print("OHHH NOO!  You had better do somthing!")
} 

#library(MASS) #Loads the MASS library, did this earlier

price_expdist <- fitdistr(saleprice, "exponential") #Fit's the sale prices to the exponential distribution

rate <- price_expdist$estimate #This is the optimal lambda from above

set.seed(42) #Sets the seed so you see what I see in this next part

exp_samples <- rexp(1000,rate) #Takes 1000 samples from the exponental distribution of sale price

par(mfrow = c(1,2)) #Makes side by side plots
hist(exp_samples, breaks = 50) #Histogram of the exponential samples
hist(saleprice, breaks = 50)  #Histogram of the origional data


As you can see, they are different.  The exponential samples weigh heavily towards 0, which would be unlikely for housing prices in most situations.  The scales are similar, although the exponential distribution has a much wider spread.

Now let's find the 5th and 95th percentiles of *the* exponential pdf using the cumulative distribution function (using our rate parameter).


In [ ]:
qexp(0.05, rate) #The 5th percentile
qexp(0.95, rate) #The 95th percentile


Interesting, but let's also compare it to the quantiles of the numbers we sampled from our exponential distribution

In [ ]:
quantile(exp_samples, c(0.05, 0.95))

Pretty close.  As we take more samples (samples -> $\infty$) we should expect the theoretical quantiles to match the actual ones, because the samples here are derived directly from the theoretical distribution.

Now let's find the Now let's find the 95% confidence interval around the mean for our empirical data (from the original dataset).  We're assuming normality, so we should be able to get this from the t.test function.


In [ ]:
t.test(saleprice, conf.level = 0.95)$conf.int

In other words, we're 95% confident that the mean Sale Price of the population at large is between about $176,842 and 185,000.

Now let's find the empirical 5th and 95th percentiles for our actual data


In [ ]:
quantile(saleprice, c(0.05,0.95))

Considering the mean is $180,921, I think we can say that this distribution is right skewed.  But what would the quantiles look like if they were coming from a normal distribution?

In [ ]:
qnorm(0.05, mean = mean(saleprice), sd= sd(saleprice))
qnorm(0.5, mean = mean(saleprice), sd= sd(saleprice))
qnorm(0.95, mean = mean(saleprice), sd= sd(saleprice))


Those percentiles would about evenly straddle the median, and the mean would be similar to the median.  We can pretty well guess that our 'SalePrice' data isn't coming from a population with a normal distribution for sale price.  Worse yet, however, would be classifying our data as exponential; those statistics are even farther off.


**10 points.  Modeling.  Build some type of multiple regression  model and submit your model to the competition board.  Provide your complete model summary and results with analysis.  Report your Kaggle.com user name and score.**

There are of ways we could do this.  I know there are neater, fancier tactics for narrowing down on a good model, but I'm going go with the tried and true "use it all then delete variables until it looks nice" method.

First though, we need to clean things up a bit.  Lot's of NA's and whatnot in here.


In [ ]:
df <- read.csv('../input/house-prices-advanced-regression-techniques/train.csv')

#Going to make a function for this so we can do the same to the testing set
#Note: this was built with the training and testing sets' NAs in mind.  If one were to use other data, this would need to be more comprehensive in it's handling of NAs and other undesireable cases.
clean <- function(df){

    df$Id <- NULL # Get rid of the ID

    df$LotFrontage[is.na(df$LotFrontage)]<- 0 #If it doesn't have frontage then set to 0

    levels(df$Alley) <- c(levels(df$Alley), "None")  #If no alley then make a new factor level, 'None'
    df$Alley[is.na(df$Alley)] <- 'None'

    df$Utilities <- NULL  #all but one has utilties

    df$Neighborhood <- NULL #Too many / won't generalize well

    df$HouseStyle <- NULL #Not going to generalize well
    
    df$MasVnrType[is.na(df$MasVnrType)] <- "None" #If not available then change to none (likely)
    
    df$MasVnrArea[is.na(df$MasVnrArea)] <- 0  #Not likely to have any
 
    levels(df$BsmtQual) <- c(levels(df$BsmtQual), "None")
    df$BsmtQual[is.na(df$BsmtQual)] <- "None" #Na is decribed in the metadata as no basement
    levels(df$BsmtCond) <- c(levels(df$BsmtCond), "None")
    df$BsmtCond[is.na(df$BsmtCond)] <- "None" #Na is decribed in the metadata as no basement
    levels(df$BsmtExposure) <- c(levels(df$BsmtExposure), "None")
    df$BsmtExposure[is.na(df$BsmtExposure)] <- "None" #Na is decribed in the metadata as no basement
    levels(df$BsmtFinType1) <- c(levels(df$BsmtFinType1), "None")
    df$BsmtFinType1[is.na(df$BsmtFinType1)] <- "None" #Na is decribed in the metadata as no basement
    levels(df$BsmtFinType2) <- c(levels(df$BsmtFinType2), "None")
    df$BsmtFinType2[is.na(df$BsmtFinType2)] <- "None" #Na is decribed in the metadata as no basement
    
    df$Electrical[is.na(df$Electrical)] <- 'SBrkr' #Makes the one instance of this the typical value
    
    levels(df$FireplaceQu) <- c(levels(df$FireplaceQu), "None") #Na is no fireplace
    df$FireplaceQu[is.na(df$FireplaceQu)] <- 'None'
     
    levels(df$GarageType) <- c(levels(df$GarageType), "None") #Na is no garage
    df$GarageType[is.na(df$GarageType)] <- 'None'
    df$GarageYrBlt[is.na(df$GarageYrBlt)] <- median(df$GarageYrBlt, na.rm = TRUE) #No great way to do this, but there's only 81 instances of NA's here in the training set
    levels(df$GarageFinish) <- c(levels(df$GarageFinish), "None") #Na is no garage
    df$GarageFinish[is.na(df$GarageFinish)] <- 'None'
    levels(df$GarageQual) <- c(levels(df$GarageQual), "None") #Na is no garage
    df$GarageQual[is.na(df$GarageQual)] <- 'None'
    levels(df$GarageCond) <- c(levels(df$GarageCond), "None") #Na is no garage
    df$GarageCond[is.na(df$GarageCond)] <- 'None'
    
    df$PoolArea <- NULL # Are few pools in training data
    df$PoolQC <- NULL
    
    df$MiscFeature <- NULL #Not enough sheds to go around
    df$MiscVal <- NULL
    
    levels(df$Fence) <- c(levels(df$Fence), "None") #Na is no fence
    df$Fence[is.na(df$Fence)] <- 'None'
    
    df$MoSold <- as.factor(df$MoSold) #This is supposed to be categorical data
    
    df$YrSold <- as.factor(df$YrSold) #Will also work well as categorical data
    
    
    df$MSZoning[is.na(df$MSZoning)] <- 'RL' #Makes instances of this the typical value
    
    
    
    df$Exterior1st[is.na(df$Exterior1st)] <- 'VinylSd'#Makes instances of these the typical value
    df$Exterior2nd[is.na(df$Exterior2nd)] <- 'VinylSd'
    
    df$BsmtFinSF1[is.na(df$BsmtFinSF1)] <- 0  #These NAs mean no basement, so we'll change the various footages to 0
    df$BsmtFinSF2[is.na(df$BsmtFinSF2)] <- 0
    df$BsmtUnfSF[is.na(df$BsmtUnfSF)] <- 0
    df$TotalBsmtSF[is.na(df$TotalBsmtSF)] <- 0
    df$BsmtFullBath[is.na(df$BsmtFullBath)] <- 0
    df$BsmtHalfBath[is.na(df$BsmtHalfBath)] <- 0
    
    df$KitchenQual[is.na(df$KitchenQual)] <- 'TA'#Makes instances of these the typical value
    
    df$Functional[is.na(df$Functional)] <- 'Typ' #Makes instances of these the typical value
    
    df$GarageCars[is.na(df$GarageCars)] <- 0
    df$GarageArea[is.na(df$GarageArea)] <- 0
    
    df$SaleType[is.na(df$SaleType)] <- 'WD' #Makes the one instance of this the typical value
    
    
    return(df) 
}

df <- clean(df)

summary(df)

Now that's done, we can try out the model that has it all.

In [ ]:
model <- lm(SalePrice ~., data = df)
str(df)
summary(model)

Adjusted R-squared of 0.9047; pretty good assuming it's not overfitting, which it may very well be given how many variables we're using.  Ordinarily one could split off a validation set to verify that, but Kaggle let's you submit your answers a bunch of times, so we'll just wait to test it out on the test set.

You might notice a few NA coefficients to in the summary above.  I believe R is doing this because those terms can be estimated by combinations of other terms.  Notably, NA coefficients tend to occur on the last category of a categorical variable, (e.g. basement condition), which may be summarized by the preceding coefficients from other categories.  However, NA does appear on basement area as well, which is numeric, and I'm not sure why that is; it was found highly predictive in the EDA.

Also, you'll probably notice there's a lot there that just really doesn't do much for the model.  Now's the part where we clean it up a bit by getting rid of the independent variables that don't do much.  For this, I'm going to employ stepwise regression (a somewhat controversial practice, I've read), which will try to eliminate some of the variables based on a specific criterion.  In this case, we'll use stepAIC, which here will select fewer variables by attempting to minimize the AIC score; in essence, this tries to cut down on variables while keeping the adjusted r-squared similar.



In [ ]:
step <- stepAIC(model, direction="backward", trace=FALSE)

In [ ]:
summary(step)


Still not exactly pretty, but that reduced the terms down to 112 while keeping the adjusted r-squared about the same.

Let's take a closer look at how the model behaves

In [ ]:
plot(model)

Starting with residuals vs fitted, we can see that for most values, the residuals are distributed fairly normally.  However, as homes get higher in price, the scale of residuals grows.  I'm thinking this could be due to a relative shortage of homes in this dataset worth over $300,000 compared to cheaper homes; maybe the model just learned how to predict the lower valued homes better.  A small curve in residuals is also indicated, which here means it may be somewhat overvaluing the very cheap homes.  We didn't use any squared terms in the model for the sake of simplicity, but those may have helped.  Overall though, the residuals seem fairly normally distributed.

As for the Q-Q plot, there's a noticeable 'wiggle' to it.  The heavy tails suggest more extreme than we would normally (pun intended) have.  However, most values still lie on the line.

The scale location plot doesn't look that great.  That the line curves like that indicated non-linearity in the model, and indicated some heteroskedasticity.

In the residuals vs leverage plot, we only have two influential cases (the ones behind the dashed lines), but these don't throw off the model too much as indicated by the horizontal red line.

Overall, this tells us that the model could definitely be improved, but should work for our purposes.

Now, let's make our predictions.


In [ ]:
testdf <- read.csv('../input/house-prices-advanced-regression-techniques/test.csv') #Reads in the test data

testID <- testdf$Id #Set's the IDs aside

testdf <- clean(testdf)  #Cleans it just like the training data

summary(testdf)

In [ ]:
predictions <- predict(model, testdf) #Makes the predictions

sum(is.na(predictions)) #Checks for NAs

head(predictions)

Awesome.  Now we just need to format our predictions and submit.  

There are definitely better, cleaner models that could be made; this is an example of brute forcing it with most the variables you have at hand.

This had a root mean squared logarithmic error of about 0.47, so it's not the best model.  I think it overfit quite a bit.  Improvements in variable selection and adding an exponent or two might help improve it.


In [ ]:
#formats the data and outputs it to a csv

output = data.frame(testID, predictions)
colnames(output) = c("Id", "SalePrice")
write.csv(output, "Predictions.csv", row.names=FALSE)
